# Load and use Mounted Azure File
 
This notebook aims to show and test how to properly use some file stored in cloud blob storages like s3, azure storage, etc.

In [ ]:
# stdlib
import os

# syft absolute
import syft as sy
from syft import ActionObject

## Connect with domain

In [ ]:
client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

## Register an existent file storage

**OBS**: The **AZURE_BLOB_STORAGE_KEY** environment key is a github env setting defined in repository that allow us to have access to a file storage created in Azure. This notebook only works if this envrionment is set and its value is valid.

In [ ]:
client.api.services.blob_storage.mount_azure(
    account_name="citestingstorageaccount",
    container_name="citestingcontainer",
    account_key=os.environ["AZURE_BLOB_STORAGE_KEY"],
    bucket_name="helmazurebucket",
)
blob_files = client.api.services.blob_storage.get_files_from_bucket(
    bucket_name="helmazurebucket"
)
document = [f for f in blob_files if "testfile.txt" in f.file_name][0]
x = ActionObject.from_obj(document)
x_ptr = x.send(client)

## Create and execute a job using stored files

In [ ]:
@sy.syft_function_single_use(document=x_ptr)
def print_remote_document(domain, document):
    print(document.read())
    return document.read()

In [ ]:
client.code.request_code_execution(print_remote_document)

In [ ]:
client.requests[-1].approve(approve_nested=True, disable_warnings=True)

In [ ]:
job = client.code.print_remote_document(document=x_ptr, blocking=False)